In [1]:
import pandas as pd
import numpy as np
import csv
import random as rnd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv("C:\Users\Andrea1\Desktop\project1\environment1/reglinPoli2.csv") ##insert your own path

In [138]:

X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[[df.columns[-1]]], train_size=0.75)
#print X_train.columns.values


In [141]:
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
Y_train=np.asarray(Y_train)
Y_test=np.asarray(Y_test)

# Normalizar

In [142]:
# This procedure is useful for classroom examples. For a real implementation you have to have a separete method 
# for transforming the production data so you can transform it as you get it with the fitted scaler
## The procedure returns a standardized copy of the input data
def normalize(X_train,X_test,Y_train,Y_test,do=True):

    scale_X=preprocessing.StandardScaler()
    scale_y=preprocessing.StandardScaler()
    
    train_X=np.copy(X_train)
    train_y=np.copy(Y_train)
    test_X=np.copy(X_test)
    test_y=np.copy(Y_test)
    if do:
        scale_X.fit(train_X)
        scale_y.fit(train_y)
        train_X=scale_X.transform(train_X)
        train_y=scale_y.transform(train_y)
        test_X=scale_X.transform(test_X)
        test_y=scale_y.transform(test_y)
    return train_X,test_X, train_y, test_y

# Y Estimada

In [143]:
## Transfer function
def salida(w,X):
    return X.dot(w[1:]) +w[0]

# Ajuste de W's

In [144]:
## Training function
def entrena(X,y,w,la=0.0,eta=0.01):
    for i in range(len(X)):
        error=y[i]-salida(w,X[i])
        w[0]=w[0]+eta*(error)
        w[1:]=w[1:]+eta*(error*X[i])-la*w[1:]
    return w


# Calcula Error

In [145]:
def calcError(X,y,w,w0):
    return np.mean((X.dot(w)+w0-y)**2)

# Cross Validation

In [146]:
# Para cierta lambda, dada la particion
def ei(Xmat,Yvec,lam,winic,Xtest,Ytest):  #Xmat son todos menos 1 bloque. Xtest es el otro bloque
        train_X,test_X, train_y, test_y=normalize(Xmat,Xtest,Yvec,Ytest)
        wi=entrena(train_X,train_y,winic,lam)
        ei=calcError(test_X,test_y.flatten(),wi[1:],wi[0])
       
        return ei


In [147]:
#La i sera el grupo test, para los primeros 2 parametros se tienen que dar todos menos i
index=np.array([rnd.randint(0,9) for i in range(len(X_train))])
def PromedioErrorParaLambda(X_train,Y_train,lam):
    errtot=0
    for i in range(0,10):
        winic=np.array([np.random.rand() for j in range(0, len(X_train[index!=i][i])+1)])
        errtot=errtot+ei(X_train[index!=i],Y_train[index!=i],lam,winic,X_train[index==i],Y_train[index==i])
    prom_ei= errtot/10   
    return prom_ei

In [152]:
lam=[0,.01,.05,.07,1]

In [157]:
for i in lam:
    print "Para lambda= "+str(i)+"    el error promedio es: "+str(PromedioErrorParaLambda(X_train,Y_train,i))

Para lambda= 0    el error promedio es: 0.57844198649
Para lambda= 0.01    el error promedio es: 0.687627448073
Para lambda= 0.05    el error promedio es: 0.915156837893
Para lambda= 0.07    el error promedio es: 0.947341014131
Para lambda= 1    el error promedio es: 1.02461843673


# Se elige lambda tal que minimice el error, esto es lambda=0